In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, IsolationForest, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import hdbscan
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [2]:
data = pd.read_csv("train.csv")
data.head()
testData = pd.read_csv("test.csv")
testData.head()

,ID,n0,n1,n2,n3,n4,n5,n6,n7,n8,...,n4086,n4087,n4088,n4089,n4090,n4091,n4092,n4093,n4094,n4095
0,0,0.0,0.000000,0.908889,0.251257,0.662262,0.042495,0.000000,0,0.964784,...,0.694072,1.146161,1.483842,0.717836,0.472616,0.000000,0.488022,0.0,0.0,0.655670
1,1,0.0,0.000000,1.191055,0.407350,0.441898,0.000000,0.334858,0,0.295357,...,0.273436,1.466932,0.940850,0.470344,1.032085,0.000000,0.654070,0.0,0.0,0.614493
2,2,0.0,0.261903,0.992782,0.301102,0.636006,0.009558,0.009448,0,0.974949,...,0.000000,0.769983,0.834360,0.369656,1.000858,0.431571,0.361993,0.0,0.0,0.392158
3,3,0.0,0.000000,1.352401,0.346003,0.401412,0.000000,0.000000,0,0.450667,...,0.339935,1.325595,0.981124,0.486731,0.747392,0.000000,0.300671,0.0,0.0,0.628365
4,4,0.0,0.000000,1.114281,0.696140,0.121505,0.000000,0.000000,0,0.591384,...,0.093661,0.875113,0.360689,0.659230,0.546044,0.000000,0.427255,0.0,0.0,0.835671


In [3]:
x = data.iloc[:, 1:-1].values
xTest = testData.iloc[:, 1:].values
xFeatures = np.array(x, dtype=float)
xTest = np.array(xTest, dtype=float)
xLabels = data.iloc[:, -1].values
xLabels = np.array(xLabels)

In [4]:
# scaler = StandardScaler()
# pca = PCA(n_components=200)
# p1 = pca.fit(xFeatures)
# x_processed = p1.transform(xFeatures)
# x_test_processed = p1.transform(xTest)
# print(x_processed.shape, x_test_processed.shape)

# lda = LDA(n_components=19, priors=None, shrinkage=None, solver='svd',
#   store_covariance=False, tol=0.0001)
# lda1 = lda.fit(x_processed, xLabels)
# xl_processed = lda1.transform(x_processed)
# print(xl_processed.shape)
# x_test_processed = lda1.transform(x_test_processed)
# print(x_test_processed.shape)
xl_processed = x
x_test_processed = xTest

In [5]:
# isolation_forest = IsolationForest(n_estimators=500, contamination=0.1)
# isolation_forest.fit(xl_processed)
# indices = np.where(isolation_forest.predict(xl_processed) != -1)[0]
# x_transformed = xl_processed[indices]
# y_transformed = xLabels[indices]
x_transformed = xl_processed
y_transformed = xLabels

#clustering
clusterer1 = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=20)
clabels = clusterer1.fit_predict(x_transformed)
print(set(clabels))
print(list(set(clabels)))
cluster_centroids = []

for i in list(set(clabels)):
    cluster_centroids.append(np.mean(x_transformed[clabels == i], axis=0))
cluster_centroids = np.array(cluster_centroids, dtype='float')
# clusterer2 = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=20)
# cluster_labels2 = clusterer2.fit_predict(x_test_processed)
# x_test_clustered = np.concatenate((x_test_processed, cluster_labels2.reshape(-1, 1)), axis=1)

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=len(cluster_centroids), init=cluster_centroids, n_init='auto')
kmeans1 = kmeans.fit(x_transformed)
cluster_labels1 = kmeans1.predict(x_transformed)
x_clustered = np.concatenate((x_transformed, cluster_labels1.reshape(-1, 1)), axis=1)
cluster_labels2 = kmeans1.predict(x_test_processed)
x_test_clustered = np.concatenate((x_test_processed, cluster_labels2.reshape(-1, 1)), axis=1)
# print(x_test_clustered.shape)
# print(set(cluster_labels1))

# print(x_test_clustered.shape)
# x_train, x_validate, y_train, y_validate = train_test_split(x_clustered, y_transformed, test_size=0.3, random_state=1)
# print(set(x_test_clustered[:, -1]))
#classification
rf = RandomForestClassifier(n_estimators=1000, max_depth=17)

{0, 1, 2, 3, 4, -1}
[0, 1, 2, 3, 4, -1]


c:\Users\rajad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:878: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.init == "k-means++":
c:\Users\rajad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1362: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params_vs_input(X, default_n_init=10)


In [6]:
print(np.mean(cross_val_score(rf, x_clustered, y_transformed, cv=5)))

KeyboardInterrupt: 

In [ ]:
print(x_test_clustered.shape, x_clustered.shape, y_transformed.shape)
rf.fit(x_clustered, y_transformed)
y_pred = rf.predict(x_test_clustered)

ids = range(415)
# print(predicted_categories)
previous_submission = pd.read_csv("submission.csv")
previous_submission.to_csv('prev.csv', index=False)
results = pd.DataFrame({'ID': ids, 'Category': y_pred})
results.to_csv('submission.csv', index=False)

(415, 20) (1216, 20) (1216,)
